<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Usando la API para obtener datos sobre personas desaparecidas del RNPDNO


<p> Julio Waissman Vilanova </p>



<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/RNPDNO-API.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

In [1]:
import os
import sys
import requests
import datetime

import pandas as pd
import json


## Calentando motores

Para descargar los datos, vamos a consultar directamente la base de datos pública del [Registro Nacional de Personas Desaparecidas y No Localizadas (RNPDNO)](https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index).

El RNPDNO no tiene una API tal cual, sin embargo, [Pablo Reyes Moctezuma](https://github.com/pablorm296) encontró una manera de extraer la información usando la librería `request` de python. La API que, me imagino, el extrajo a punta de prueba y error la documento en [este archivo en markdown](https://github.com/pablorm296/ScrapperRNPDNO/blob/master/Test/API.md). Un chambón.

Vamos air la usando poco a poco, empecemos por tratar de encontrar en el catálogo los indices de estados, municipios y colonias.

In [2]:
API_HOST = "https://versionpublicarnpdno.segob.gob.mx/"
API_SOCIODEOGRAFICOS_ROOT = "Sociodemografico/"
API_CATALAGO_ROOT = "Catalogo/"

ENDPOINT_CATALOGO_EDO = "Estados/"
ENDPOINT_CATALOGO_MUN = "Municipios/"
ENDPOINT_CATALOGO_COL = "Colonias/"

# Before doing anything, we must make a dummy request to the index in order to get the propper cookies
main_session = requests.Session()
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index")
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/ContextoGeneral")

<Response [200]>

Los identificadores de los estados:

In [3]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_EDO

r = main_session.post(TARGET_URL)
estados_id = pd.json_normalize(r.json(),)
estados_id.columns = ['Valor', 'Estado']
estados_id

,Valor,Estado
0,0,--TODOS--
1,1,AGUASCALIENTES
2,2,BAJA CALIFORNIA
3,3,BAJA CALIFORNIA SUR
4,4,CAMPECHE
5,7,CHIAPAS
6,8,CHIHUAHUA
7,9,CIUDAD DE MEXICO
8,5,COAHUILA
9,6,COLIMA


y ahora los municipios de Sonora

In [4]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_MUN
DATA = {"idEstado": "26"}

r = main_session.post(TARGET_URL, data = DATA)
mun_son_id = pd.json_normalize(r.json())
mun_son_id.columns = ['Valor', 'Municipio']
mun_son_id.head(20)

,Valor,Municipio
0,0,--TODOS--
1,1,ACONCHI
2,2,AGUA PRIETA
3,3,ALAMOS
4,4,ALTAR
5,5,ARIVECHI
6,6,ARIZPE
7,7,ATIL
8,8,BACADÉHUACHI
9,9,BACANORA


y por último los identificadores de las colonias del municipio de Hermosillo

In [5]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_COL
DATA = {"idEstado": "26", "idMunicipio": "30"}

r = main_session.post(TARGET_URL, data = DATA)
col_hmo_id = pd.json_normalize(r.json())
col_hmo_id.columns = ['Valor', 'Municipio']
col_hmo_id

,Valor,Municipio
0,0,--TODAS--
1,347025,22 DE SEPTIEMBRE
2,347004,26 DE OCTUBRE
3,347026,4 DE MARZO
4,347027,4 OLIVOS
...,...,...
631,347358,VISTA DEL LAGO
632,347359,Y
633,347582,ZACATON
634,347384,ZAMORA


## Sociodemográficos totales

`Con este `endpoint` se pueden consultar resúmenes generales de la información que se pide. recuerda de revisar los catálogos.

Hay dos variables cuyos valores posibles son los siguientes:

**idEstatusVictima**:
- "0" PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS
- "2" PERSONAS LOCALIZADAS CON VIDA
- "3" PERSONAS LOCALIZADAS SIN VIDA- "4" PERSONAS DESAPARECIDAS
- "5" PERSONAS NO LOCALIZADAS
- "6" PERSONAS LOCALIZADAS
- "7" PERSONAS DESAPARECIDAS Y NO LOCALIZADAS

**idHipotesisNoLocalizacion**:
- "0" --TODAS--
- "1" ACCIDENTE
- "2" CATÁSTROFE
- "3" NO LOCALIZACIÓN VOLUNTARIA
- "4" NO LOCALIZACIÓN INVOLUNTARIA
- "5" SE DESCONOCE

Veamos como funciona pidiendo información de Sonora y de Hermosillo. Empecemos por Sonora

In [6]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT  + "Totales"

DATA = {
  "titulo":"",
  "subtitulo": "",
    "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
  "idDelito":"0"
}
r = main_session.post(TARGET_URL, json = DATA)

resumen_sonora = pd.json_normalize(r.json()).T
resumen_sonora.columns = ['Valor']

resumen_sonora


,Valor
TotalGlobal,"7,960"
TotalDesaparecidos,"4,770"
TotalLocalizados,"3,190"
PorcentajeDesaparecidos,59.92 %
PorcentajeLocalizados,40.08 %
TotalSoloDesaparecidos,"4,730"
TotalSoloNoLocalizados,40
PorcentajeSoloDesaparecidos,99.16 %
PorcentajeSoloNoLocalizados,0.84 %
TotalLocalizadosCV,"2,929"


### Ejercicio

Probar con diferentes consultas y tratar de inferir los valores que pueden tomar (o buscarlas en la documentación de la API) las diferentes variables que pueden servir para encontrar búsquedas más específicas.

Por ejemplo, ¿Como podríamos consultar las estadísticas sobre mujeres desaparecidas en el municipio de Cajeme?

Para esto vamos a necesitar usar la parte siguiente de la libreta, hacemos la consulta con el id del municipio de Cajeme y usando todas las colonias. Hecho esto, sumamos las mujeres de desaparecidas de todas las colonias, incluyendo las no especificadas, para obtener un total.

In [7]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR COLONIAS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":"18",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)

res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
por_colonia = pd.DataFrame(datos)

mujeres = por_colonia['Mujer'].sum()
print(f'Las mujeres desaparecidas en Cajeme son: {mujeres}')

Las mujeres desaparecidas en Cajeme son: 265


## Personas desaparecidas por sexo y colonia

El `endpoint` **BarChartSexoColonia** está diseñado para generar gráficas, pero nos permite extraer información, si la sabemos formatear.

Vamos viendo un ejemplo:


In [8]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR MUNICIPIOS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":"30",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)


res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Colonia'] = res['XAxisCategories']
por_colonia = pd.DataFrame(datos)
por_colonia.index = por_colonia.Colonia

por_colonia


,Hombre,Mujer,Indeterminado,Colonia
Colonia,,,,
VILLA HERMOSA,0,4,0,VILLA HERMOSA
MIGUEL ALEMÁN CENTRO,11,6,0,MIGUEL ALEMÁN CENTRO
QUINTA EMILIA,2,0,0,QUINTA EMILIA
ARBOLEDAS,1,1,0,ARBOLEDAS
SAHUARO INDECO,0,2,0,SAHUARO INDECO
UNIVERSIDAD,1,0,0,UNIVERSIDAD
JESÚS GARCIA,7,3,0,JESÚS GARCIA
MESA DEL SERI,1,1,0,MESA DEL SERI
NUEVA PALMIRA,1,0,0,NUEVA PALMIRA


### Ejercicio

¿Como podemos sacar lo que pasa en todo el estado, por municipios y por colonias? Intentalo.

Suponemos que existe un ENDPOINT de BarChart que determine sexo y municipio, Modificamos el TARGET.

In [ ]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoMunicipio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR MUNICIPIOS - SONORA",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)

res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Municipio'] = res['XAxisCategories']
#datos
por_municipio = pd.DataFrame(datos)
por_municipio.index = por_municipio.Municipio

por_municipio

,Hombre,Mujer,Indeterminado,Municipio
Municipio,,,,
HERMOSILLO,1231,722,0,HERMOSILLO
NOGALES,669,403,0,NOGALES
CAJEME,649,317,0,CAJEME
GUAYMAS,331,93,0,GUAYMAS
AGUA PRIETA,320,182,0,AGUA PRIETA
CABORCA,286,83,0,CABORCA
PUERTO PEÑASCO,248,109,0,PUERTO PEÑASCO
SAN LUIS RÍO COLORADO,225,168,1,SAN LUIS RÍO COLORADO
NAVOJOA,177,64,0,NAVOJOA


Haremos unas graficas basicas que nos hace colab

In [ ]:
por_municipio

,Hombre,Mujer,Indeterminado,Municipio
0,1231,722,0,HERMOSILLO
1,669,403,0,NOGALES
2,649,317,0,CAJEME
3,331,93,0,GUAYMAS
4,320,182,0,AGUA PRIETA
5,286,83,0,CABORCA
6,248,109,0,PUERTO PEÑASCO
7,225,168,1,SAN LUIS RÍO COLORADO
8,177,64,0,NAVOJOA
9,171,24,0,GENERAL PLUTARCO ELÍAS CALLES


from matplotlib import pyplot as plt
por_municipio['Hombre'].plot(kind='hist', bins=20, title='Hombre')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
por_municipio['Mujer'].plot(kind='hist', bins=20, title='Mujer')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
por_municipio['Indeterminado'].plot(kind='hist', bins=20, title='Indeterminado')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
por_municipio.plot(kind='scatter', x='Hombre', y='Mujer', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
por_municipio.plot(kind='scatter', x='Mujer', y='Indeterminado', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
por_municipio['Hombre'].plot(kind='line', figsize=(8, 4), title='Hombre')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
por_municipio['Mujer'].plot(kind='line', figsize=(8, 4), title='Mujer')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
por_municipio['Indeterminado'].plot(kind='line', figsize=(8, 4), title='Indeterminado')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
por_municipio.describe() #Como que los datos que extrae siempre son conjuntos de 30 datos

,Hombre,Mujer,Indeterminado
count,30.000000,30.000000,30.000000
mean,177.366667,82.633333,0.200000
std,261.392093,153.245236,0.924755
min,12.000000,2.000000,0.000000
25%,30.000000,8.000000,0.000000
50%,88.000000,22.500000,0.000000
75%,213.000000,78.250000,0.000000
max,1231.000000,722.000000,5.000000


In [ ]:
Aqui intente sacarlos de manera independiente:

In [ ]:
TARGET_URL_MUNICIPIOS = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoMunicipio"
TARGET_URL_COLONIAS = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

# Parámetros de la solicitud
DATA = {
    "titulo": "PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
    "subtitulo": "Por Municipios y colonias",
    "idEstado": "26",  # Sonora
    "idMunicipio": "0",  # 0 para todos los municipios
    "idColonia": "0",    # 0 para todas las colonias
    "idEstatusVictima": "0",
    "idHipotesisNoLocalizacion": "0",
    "idDelito": "0",
    "fechaInicio": "",
    "fechaFin": "",
    "mostrarFechaNula": "0",
    "idNacionalidad": "0",
    "edadInicio": "",
    "edadFin": "",
    "mostrarEdadNula": "0",
    "idHipotesis": "",
    "idMedioConocimiento": "",
    "idCircunstancia": "",
    "tieneDiscapacidad": "",
    "idTipoDiscapacidad": "0",
    "idEtnia": "0",
    "idLengua": "0",
    "idReligion": "",
    "esMigrante": "",
    "idEstatusMigratorio": "0",
    "esLgbttti": "",
    "esServidorPublico": "",
    "esDefensorDH": "",
    "esPeriodista": "",
    "esSindicalista": "",
    "esONG": ""
}

# Realiza la solicitud al API para obtener datos por municipios
r_municipio = main_session.post(TARGET_URL_MUNICIPIOS, json=DATA)
res_municipio = r_municipio.json()

# Extrae los datos de la serie y las categorías (municipios)
datos_municipios = {serie['name']: serie['data'] for serie in res_municipio['Series']}
datos_municipios['Municipio'] = res_municipio['XAxisCategories']

# Crea el DataFrame inicial para los datos por municipios
por_municipio = pd.DataFrame(datos_municipios)

# Crea un DataFrame vacío para acumular los datos por colonias
datos_totales = pd.DataFrame()

# Itera sobre cada fila del DataFrame para obtener los datos por colonia
for idx, row in por_municipio.iterrows():
    municipio_nombre = row['Municipio']
    municipio_id = idx + 1  # Ajusta según la lógica de IDs en tu API

    # Actualiza el DATA para solicitar datos específicos del municipio
    DATA['idMunicipio'] = str(municipio_id)

    # Realiza la solicitud para obtener los datos de colonias en el municipio actual
    r_colonia = main_session.post(TARGET_URL_COLONIAS, json=DATA)
    res_colonia = r_colonia.json()

    # Extrae los datos de la serie y las categorías (colonias)
    datos_colonia = {serie['name']: serie['data'] for serie in res_colonia['Series']}
    datos_colonia['Colonia'] = res_colonia['XAxisCategories']

    # Crea un DataFrame para las colonias del municipio actual
    df_colonia = pd.DataFrame(datos_colonia)
    df_colonia['Municipio'] = municipio_nombre  # Asigna el nombre del municipio a las colonias

    # Concatenar los datos de colonias con los datos generales
    datos_totales = pd.concat([datos_totales, df_colonia], ignore_index=True)

# Organiza el DataFrame final
datos_totales.set_index(['Municipio', 'Colonia'], inplace=True)

# DataFrame resultante
datos_totales


Hombre  Mujer  Indeterminado
Municipio  Colonia                                                
HERMOSILLO ACONCHI                       2.0    0.0            0.0
           SIN COLONIA DE REFERENCIA     2.0    3.0            0.0
NOGALES    VALLES DUARTE                 0.0    3.0            0.0
           BUENOS AIRES                  4.0    6.0            0.0
           PROGRESO                      3.0    3.0            0.0
...                                      ...    ...            ...
URES       4 DE MARZO                    1.0    0.0            0.0
           PASEO CASA BLANCA             1.0    0.0            0.0
           RANCHITO                      0.0    1.0            0.0
           SAHUARO FINAL                 0.0    1.0            0.0
           GALA                          1.0    1.0            0.0

[228 rows x 3 columns]

Por alguna razon, no se cual y no pude resolverlo, me cambia hermosillo por URES. :(

In [ ]:
datos_totales

Hombre  Mujer  Indeterminado
Municipio  Colonia                                                
HERMOSILLO ACONCHI                       2.0    0.0            0.0
           SIN COLONIA DE REFERENCIA     2.0    3.0            0.0
NOGALES    VALLES DUARTE                 0.0    3.0            0.0
           BUENOS AIRES                  4.0    6.0            0.0
           PROGRESO                      3.0    3.0            0.0
...                                      ...    ...            ...
URES       4 DE MARZO                    1.0    0.0            0.0
           PASEO CASA BLANCA             1.0    0.0            0.0
           RANCHITO                      0.0    1.0            0.0
           SAHUARO FINAL                 0.0    1.0            0.0
           GALA                          1.0    1.0            0.0

[228 rows x 3 columns]

from matplotlib import pyplot as plt
datos_totales['Hombre'].plot(kind='hist', bins=20, title='Hombre')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
datos_totales['Mujer'].plot(kind='hist', bins=20, title='Mujer')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
datos_totales.plot(kind='scatter', x='Hombre', y='Mujer', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Indeterminado']
  ys = series['Hombre']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = datos_totales.sort_values('Indeterminado', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Indeterminado')
_ = plt.ylabel('Hombre')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Indeterminado']
  ys = series['Mujer']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = datos_totales.sort_values('Indeterminado', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Indeterminado')
_ = plt.ylabel('Mujer')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Indeterminado']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Indeterminado'}, axis=1)
              .sort_values('Indeterminado', ascending=True))
  xs = counted['Indeterminado']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = datos_totales.sort_values('Indeterminado', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Indeterminado')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
datos_totales['Hombre'].plot(kind='line', figsize=(8, 4), title='Hombre')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
datos_totales['Mujer'].plot(kind='line', figsize=(8, 4), title='Mujer')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
datos_totales.describe()

,Hombre,Mujer,Indeterminado
count,228.000000,228.000000,228.0
mean,8.820175,3.504386,0.0
std,45.417671,17.804493,0.0
min,0.000000,0.000000,0.0
25%,1.000000,0.000000,0.0
50%,1.000000,1.000000,0.0
75%,2.000000,1.000000,0.0
max,527.000000,234.000000,0.0


## Información por sexo y por año

Tambien se puede encontrar información por sexo y por año utilizando otro `endpoint`: **AreaChartSexoAnio**

Sin mas choro, vamos a ver como se usa, otra vez con el estado de Sonora:

In [ ]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR AÑO EN EL ESTADO DE SONORA",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "idEstatusVictima":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}

r = main_session.post(TARGET_URL, json = DATA)

res = r.json()

datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Fecha'] = res['XAxisCategories']

por_fecha = pd.DataFrame(datos)
por_fecha['Fecha'] = pd.to_numeric(por_fecha.Fecha, errors='coerce')
por_fecha.index = por_fecha.Fecha

por_fecha


,Hombre,Mujer,Indeterminado,Fecha
Fecha,,,,
NaN,127,45,5,NaN
1974.0,3,0,0,1974.0
1977.0,1,0,0,1977.0
1978.0,2,0,0,1978.0
1980.0,1,0,0,1980.0
1981.0,7,1,0,1981.0
1982.0,1,0,0,1982.0
1989.0,1,0,0,1989.0
1994.0,2,0,0,1994.0


### Ejercicio

¿Se puede hacer por municipio? ¿En forma programática? ¿Para algun caso especial? Intentalo

Aqui agrupo cada municipio con anios que aparezcan con desaparecidos.

In [ ]:
TARGET_URL_MUNICIPIOS = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoMunicipio"
TARGET_URL_FECHAS = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

# Parámetros base de la solicitud
DATA_BASE = {
    "titulo": "PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
    "subtitulo": "Por Municipios y colonias",
    "idEstado": "26",  # Sonora
    "idMunicipio": "0",  # 0 para todos los municipios
    "idColonia": "0",
    "idEstatusVictima": "0",
    "idHipotesisNoLocalizacion": "0",
    "idDelito": "0",
    "fechaInicio": "",
    "fechaFin": "",
    "mostrarFechaNula": "0",
    "idNacionalidad": "0",
    "edadInicio": "",
    "edadFin": "",
    "mostrarEdadNula": "0",
    "idHipotesis": "",
    "idMedioConocimiento": "",
    "idCircunstancia": "",
    "tieneDiscapacidad": "",
    "idTipoDiscapacidad": "0",
    "idEtnia": "0",
    "idLengua": "0",
    "idReligion": "",
    "esMigrante": "",
    "idEstatusMigratorio": "0",
    "esLgbttti": "",
    "esServidorPublico": "",
    "esDefensorDH": "",
    "esPeriodista": "",
    "esSindicalista": "",
    "esONG": ""
}

# Solicitud al API para obtener datos por municipios
r_municipio = main_session.post(TARGET_URL_MUNICIPIOS, json=DATA_BASE)
res_municipio = r_municipio.json()

# Extrae los nombres de los municipios
municipios = res_municipio['XAxisCategories']

# DataFrame vacío para acumular los datos por municipio y fecha
datos_totales = pd.DataFrame()

# Itera sobre cada municipio para obtener los datos por año
for i, municipio in enumerate(municipios):
    # Actualiza el DATA para solicitar datos específicos del municipio
    DATA_MUNICIPIO = DATA_BASE.copy()
    DATA_MUNICIPIO['idMunicipio'] = str(i + 1)  # Asume que los IDs de municipios en la API son consecutivos

    # Realiza la solicitud para obtener los datos por año en el municipio actual
    r_fecha = main_session.post(TARGET_URL_FECHAS, json=DATA_MUNICIPIO)
    res_fecha = r_fecha.json()

    # Extrae los datos de la serie y las categorías (años)
    datos_fecha = {serie['name']: serie['data'] for serie in res_fecha['Series']}
    datos_fecha['Fecha'] = res_fecha['XAxisCategories']

    # Crea un DataFrame para los datos del municipio actual
    df_fecha = pd.DataFrame(datos_fecha)
    df_fecha['Municipio'] = municipio  # Asigna el nombre del municipio a las filas

    # Concatenar los datos del municipio con los datos generales
    datos_totales = pd.concat([datos_totales, df_fecha], ignore_index=True)

# Organiza el DataFrame final
datos_totales.set_index(['Municipio', 'Fecha'], inplace=True)

# DataFrame resultante
datos_totales


Hombre  Mujer  Indeterminado
Municipio  Fecha                                                       
HERMOSILLO 1.CIFRA SIN  AÑO DE REFERENCIA     1.0    0.0            0.0
           2013                               0.0    1.0            0.0
           2016                               1.0    0.0            0.0
           2017                               2.0    0.0            0.0
           2019                               0.0    1.0            0.0
...                                           ...    ...            ...
URES       2020                             142.0   17.0            0.0
           2021                              26.0    9.0            0.0
           2022                              30.0    8.0            0.0
           2023                              46.0   15.0            0.0
           2024                             141.0   47.0            0.0

[260 rows x 3 columns]

In [ ]:
datos_totales

Hombre  Mujer  Indeterminado
Municipio  Fecha                                                       
HERMOSILLO 1.CIFRA SIN  AÑO DE REFERENCIA     1.0    0.0            0.0
           2013                               0.0    1.0            0.0
           2016                               1.0    0.0            0.0
           2017                               2.0    0.0            0.0
           2019                               0.0    1.0            0.0
...                                           ...    ...            ...
URES       2020                             142.0   17.0            0.0
           2021                              26.0    9.0            0.0
           2022                              30.0    8.0            0.0
           2023                              46.0   15.0            0.0
           2024                             141.0   47.0            0.0

[260 rows x 3 columns]

from matplotlib import pyplot as plt
datos_totales['Hombre'].plot(kind='hist', bins=20, title='Hombre')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
datos_totales['Mujer'].plot(kind='hist', bins=20, title='Mujer')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
datos_totales.plot(kind='scatter', x='Hombre', y='Mujer', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Indeterminado']
  ys = series['Hombre']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = datos_totales.sort_values('Indeterminado', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Indeterminado')
_ = plt.ylabel('Hombre')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Indeterminado']
  ys = series['Mujer']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = datos_totales.sort_values('Indeterminado', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Indeterminado')
_ = plt.ylabel('Mujer')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Indeterminado']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Indeterminado'}, axis=1)
              .sort_values('Indeterminado', ascending=True))
  xs = counted['Indeterminado']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = datos_totales.sort_values('Indeterminado', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Indeterminado')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
datos_totales['Hombre'].plot(kind='line', figsize=(8, 4), title='Hombre')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
datos_totales['Mujer'].plot(kind='line', figsize=(8, 4), title='Mujer')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
datos_totales.describe()

,Hombre,Mujer,Indeterminado
count,260.000000,260.000000,260.0
mean,12.700000,5.950000,0.0
std,25.828107,14.971833,0.0
min,0.000000,0.000000,0.0
25%,1.000000,0.000000,0.0
50%,3.000000,1.000000,0.0
75%,13.000000,4.000000,0.0
max,194.000000,140.000000,0.0


### Ejercicio

Extrae alguna información del conjunto de tados que pienses que es relevante, y explica porqué.

Lo mas relevante ya estaba, lo que se puede hacer ahora es hacer los describes de los dataframes resultantes, lo cual lo fui haciendo arriba. Tambien elaborar buenas graficas pero por cuestion de tiempo ya no pude hacer unas decentes, solo las de colab.

## Practicando a ser investigador de APIs

Ahora te pido que revises si puedes encontrar otros endpoints para recuperar mas información de las bases que no se encuentran liberadas. Puede ser en la misma página, o en blogs o revisando código. Agrega en esta libreta la documentación (o enlaces a dicha documentación) y un ejemplo de uso de una API pobremente documentada.